In [42]:
plt.close('all')

In [5]:

"""
This script contains a lot of useful functions:
- Calculating the ionization cross section using parameters from
Schreiber and Williams or Powell (see thesis for references).
- Calculate the theoretical zeta-factor and k-factor.
- Least-Square fitting of zeta-factors by fitting scaling factors
to the ionization cross section, Au contact layer and Si dead layer
of the detector.
- Plots the fitted model and detector efficiency as function of
energy and element.
- Plots the ionization cross section as a function of overvoltage.
"""
%matplotlib qt4
import numpy as np
from hyperspy.misc.elements import elements as elements_db
from hyperspy.misc.eds.utils import _get_element_and_line
from hyperspy.misc.material import mass_absorption_coefficient
import scipy
import scipy.constants
from scipy.constants import e as elementary_charge
from scipy.constants import pi as PI
import matplotlib.pyplot as plt
import matplotlib
plt.close('all')
matplotlib.rcParams.update({'font.size': 14})
# Fluorescence yield from Hubbel et al. (See Reference section of thesis)
# Fluorescence yields (K (3 < Z < 100), L (11 < Z < 100), 19 < Z < 100)
# The given fluorescence yield for L- and M-lines are the average of all subshells
fluorescence_yield_K_lines = [2.93E-04, 6.93E-04, 0.001409, 0.002575, 0.004349,
                              0.006909, 0.01045, 0.01519, 0.02133, 0.03, 0.04, 0.05, 0.06, 0.08, 0.10, 0.12,
                              0.14, 0.17, 0.20, 0.23, 0.26, 0.29, 0.32, 0.35, 0.39, 0.42, 0.45, 0.49, 0.52,
                              0.55, 0.57, 0.60, 0.63, 0.65, 0.67, 0.70, 0.72, 0.73, 0.75, 0.77, 0.78, 0.80,
                              0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.88, 0.88, 0.89, 0.90, 0.90,
                              0.91, 0.91, 0.92, 0.92, 0.93, 0.93, 0.93, 0.93, 0.94, 0.94, 0.94, 0.94, 0.95,
                              0.95, 0.95, 0.95, 0.95, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96,
                              0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97,
                              0.97, 0.97, 0.97, 0.97]
fluorescence_yield_L_lines = [2.17E-04, 3.04E-04, 4.15E-04, 5.53E-04, 7.24E-04,
                              9.30E-04, 1.18E-03, 1.47E-03, 1.81E-03, 2.21E-03, 2.68E-03, 3.21E-03, 3.81E-03,
                              4.50E-03, 5.27E-03, 6.14E-03, 7.11E-03, 8.19E-03, 9.39E-03, 1.07E-02, 1.22E-02,
                              1.38E-02, 1.55E-02, 1.74E-02, 1.95E-02, 2.18E-02, 2.42E-02, 2.63E-02, 2.85E-02,
                              3.09E-02, 3.35E-02, 3.63E-02, 3.93E-02, 4.25E-02, 4.59E-02, 4.95E-02, 5.34E-02,
                              5.75E-02, 6.18E-02, 6.65E-02, 7.14E-02, 7.65E-02, 8.20E-02, 8.77E-02, 9.38E-02,
                              1.00E-01, 1.07E-01, 1.14E-01, 1.21E-01, 1.29E-01, 1.37E-01, 1.45E-01, 1.53E-01,
                              1.63E-01, 1.72E-01, 1.82E-01, 1.92E-01, 2.02E-01, 2.12E-01, 2.23E-01, 2.34E-01,
                              2.45E-01, 2.57E-01, 2.69E-01, 2.81E-01, 2.93E-01, 3.05E-01, 3.18E-01, 3.31E-01,
                              3.43E-01, 3.56E-01, 3.69E-01, 3.82E-01, 3.95E-01, 4.09E-01, 4.22E-01, 4.35E-01,
                              4.48E-01, 4.61E-01, 4.74E-01, 4.86E-01, 4.99E-01, 5.11E-01, 5.24E-01, 5.36E-01,
                              5.48E-01, 5.60E-01, 5.72E-01, 5.83E-01, 5.95E-01]

fluorescence_yield_M_lines = [1.67E-06, 3.10E-06, 5.28E-06, 8.46E-06, 1.29E-05,
                              1.89E-05, 2.67E-05, 3.68E-05, 4.96E-05, 6.53E-05, 8.45E-05, 1.08E-04, 1.35E-04,
                              1.68E-04, 2.06E-04, 2.51E-04, 3.02E-04, 3.61E-04, 4.28E-04, 5.04E-04, 5.90E-04,
                              6.86E-04, 7.93E-04, 9.12E-04, 1.04E-03, 1.19E-03, 1.35E-03, 1.53E-03, 1.72E-03,
                              1.93E-03, 2.17E-03, 2.42E-03, 2.69E-03, 2.98E-03, 3.30E-03, 3.65E-03, 4.01E-03,
                              4.41E-03, 4.84E-03, 5.29E-03, 5.78E-03, 6.29E-03, 6.85E-03, 7.44E-03, 8.06E-03,
                              8.73E-03, 9.43E-03, 1.02E-02, 1.10E-02, 1.18E-02, 1.27E-02, 1.36E-02, 1.46E-02,
                              1.56E-02, 1.67E-02, 1.79E-02, 1.91E-02, 2.03E-02, 2.16E-02, 2.30E-02, 2.45E-02,
                              2.60E-02, 2.75E-02, 2.92E-02, 3.10E-02, 3.28E-02, 3.47E-02, 3.66E-02, 3.87E-02,
                              4.08E-02, 4.30E-02, 4.53E-02, 4.77E-02, 5.02E-02, 5.28E-02, 5.55E-02, 5.83E-02,
                              6.12E-02, 6.42E-02, 6.73E-02, 7.06E-02, 7.39E-02]


def get_fluorescence_yield(xray_line):
    """ Returns the fluorescence yield for a given X-ray line """
    element, line = xray_line.split('_')
    family = line[0]
    el_data = elements_db[element]
    z = el_data['General_properties']['Z']
    if z > 100:
        raise ValueError("No data for Z > 100")
    if family == 'K':
        if z < 3:
            raise ValueError("Error: No data for Z < 3 for K-lines")
        return fluorescence_yield_K_lines[z - 3]
    elif family == 'L':
        if z < 11:
            raise ValueError("Error: No data for Z < 11 for L-lines")
        return fluorescence_yield_L_lines[z - 11]
    elif family == 'M':
        if z < 19:
            raise ValueError("Error: No data for Z < 19 for M-lines")
        return fluorescence_yield_M_lines[z - 19]
    raise ValueError('NotavalidX - rayline!')


def schreiber_wims_cross_section(z, line_family):
    """ Returns parameters for the inner-shell ionization cross section as described by Schreiber and Wims"""

    ln_z = np.log(z)
    if line_family == 'K':
        if z <= 30:
            b = 8.874 - (8.158 * ln_z) + (2.9055 * ((ln_z) ** 2)) - (0.35778* (ln_z ** 3))
        else:
            b = 0.661
        d = 1.0667 - (0.00476 * z)
    elif line_family == 'L':
        b = 0.2704 + (0.00726 * ((ln_z) ** 3))
        d = 1
    elif line_family == 'M':
        b = 11.33 - 2.43 * ln_z
        d = 1
    else:
        raise Exception("Invalid line")
    c = 1
    return b, c, d


def powell_cross_section(line_family):
    """ Returns parameters for the inner-shell ionization cross section
    as described by Powell """

    if line_family == 'K':
        b = 0.9
        c = 0.70
    elif line_family == 'L':
        # It was not clear from Powell(1976) what the recommended values for L - lines were,
        # so uses L-lines from Powell as given in
        # "Analytical Transmission Electron Microscopy: An Introduction for Operators(2014)
        c = 0.59
        b = 0.63
    else:
        raise Exception('PowellcrosssectiononlyacceptsK - or L - lines')
    d = 1
    return b, c, d


def ionization_cross_section(xray_line, beam_energy, method, normalize=False):
    """
    Calculates the inner-shell ionization cross section in m^2
    If normalize is True, units are m^2 * eV^2
    Parameters
    ----------
    xray_line: str
    The name of the X-ray line, i.e. Al_Ka
    beam_energy: float
    Beam energy in keV
    method: str
    'sw': Schreiber-Wims
    'powell': Powell
    normalize: bool
    If True: The cross section is normalized by ionization energy
    """

    element, line = xray_line.split('_')
    line_family = line[0]
    ionization_energy = elements_db[element]['Atomic_properties']['Xray_lines'][line]['energy (keV)']
    z = elements_db[element]['General_properties']['Z']
    # Get number of electrons in shell
    if line_family == 'K':
        ns = 2
    elif line_family == 'L':
        ns = 8
    elif line_family == 'M':
        ns = 18
    else:
        raise Exception("Invalid line family: Need to be K-, L- or M-line")
    # Get ionization cross section
    if method == 'sw':
        b, c, d = schreiber_wims_cross_section(z, line_family)
    elif method == 'powell':
        b, c, d = powell_cross_section(line_family)

    U = beam_energy / ionization_energy  # overvoltage
    num = 6.4924e-24 * ns * b * np.log(c * U)
    if normalize:
        den = U ** d
    else:
        den = (ionization_energy ** 2) * (U ** d)
    Q = num / den
    return Q


def plot_Q_vs_overvoltage(xray_line, overvoltage_bounds, normalize=False, method='sw', format='r-', new_figure=True):
    element, line = xray_line.split('_')
    line_energy = elements_db[element]['Atomic_properties']['Xray_lines'][line]['energy (keV)']
    z = elements_db[element]['General_properties']['Z']
    beam_min = overvoltage_bounds[0] * line_energy
    beam_max = overvoltage_bounds[1] * line_energy
    print("beam_min:", beam_min, "beam_max:", beam_max)
    U = []
    Q = []
    for beam_energy in np.linspace(beam_min, beam_max, (beam_max - beam_min + 1) * 50):
        q = ionization_cross_section(xray_line, beam_energy, method, normalize)
        Q.append(q)
        U.append(beam_energy / line_energy)
    if new_figure:
        plt.figure()
    format='r-'
    plt.plot(U, Q, format)
    plt.xlabel('Overvoltage')
    if normalize:
        plt.ylabel('Ionization cross section(m ^ 2) * (eV ^ 2)')
    else:
        plt.ylabel('Ionization cross section(m ^ 2)')
        # Plots inner-shell ionization cross sections for both implemented methods.
# overvoltage_bounds = (1, 30)
# plot_Q_vs_overvoltage('As_Ka', overvoltage_bounds, normalize=False, new_figure=True,
#                       method='sw', format='b -')
# plot_Q_vs_overvoltage('As_Ka', overvoltage_bounds, normalize=False, new_figure=False,
#                       method='powell', format='g - -')
# plt.ylim(0)
# Detector configuration
# Thicknesses in cm
detector_data = {
    'collection_angle': 0.98,  # srad
    't_polymer': 0,  # Don't know the composition of this, therefore this term is ignored(assume windowless)
    't_Au': 1e-6,  # From metadata - from m to cm
    't_Si': 1e-5,  # From metadata from m to cm
    't_Si_2': 0.1  # From Williams & Carter
}


def calculate_theoretical_zeta_factor(xray_line, detector, beam_energy,
                                      cross_section='sw'):
    """
    Parameters
    ----------
    xray_line: str
    Name of the X-ray line
    detector: dictionary
    Dictionary containing the window thickness of
    the polymer window, gold layer, silicon dead layer
    and high energy layer.
    beam_energy: float
    Beam energy in keV
    cross_section: 'sw' or 'powell'
    If 'sw' use the ionization cross section defined by Schreiber and Wims
    (default)
    If 'powell' use the ionization cross section defined by Powell
    """

    element, line = xray_line.split('_')
    el_data = elements_db[element]
    # z = el_data['General_properties']['Z']
    atomic_weight = el_data['General_properties']['atomic_weight']
    a = el_data['Atomic_properties']['Xray_lines'][line]['weight']
    w = get_fluorescence_yield(xray_line)
    collection_angle_4pi = detector['collection_angle'] / (4 * scipy.pi)
    q = ionization_cross_section(xray_line, beam_energy, cross_section)
    detector_efficiency = estimate_detector_efficiency(xray_line, detector['t_polymer'], detector['t_Au'],detector['t_Si'], detector['t_Si_2'])
    return atomic_weight / (scipy.constants.N_A * q * w * a * collection_angle_4pi * detector_efficiency)


def _get_element_density(element):
    """ Returns density of the given element in g/cm^3 """
    return elements_db[element]['Physical_properties']['density (g/cm^3)']


def estimate_detector_efficiency(xray_line, t_polymer, t_Au, t_Si, t_Si_2):
    """ Estimates the detector efficiency. Thickness given in cm. """

    polymer_window = 1  # Don't know the composition of this, but it's thin, so assume0 - canbeimproved if window is known
    Au_contact = np.exp(-mass_absorption_coefficient('Au', xray_line) * _get_element_density('Au') * t_Au)
    Si_dead_layer = np.exp(-mass_absorption_coefficient('Si', xray_line) * _get_element_density('Si') * t_Si)
    high_energy_layer = 1 - np.exp(-mass_absorption_coefficient('Si', xray_line) * _get_element_density('Si') * t_Si_2)
    return polymer_window * Au_contact * Si_dead_layer * high_energy_layer


# Reference is usually 'Si_Ka' or 'Fe_Ka'
def calculate_theoretical_k_factor(xray_lines, detector, beam_energy):
    """
    Calculates the theoretical k-factor.
    Parameters
    ----------70
    xray_lines: list of strings
    The two X-ray lines for the k-factor
    detector: dictionary
    Thicknesses of detector windows/layers and collection angle.
    beam_energy: float
    The energy of the beam
    """

    atomic_weight = []
    res = []
    for xl in xray_lines:
        element, line = xl.split('_')
        el_data = elements_db[element]
        z = el_data['General_properties']['Z']
        atomic_weight.append(el_data['General_properties']['atomic_weight'])
        w = get_fluorescence_yield(xl)
        a = el_data['Atomic_properties']['Xray_lines'][line]['weight']
        q = ionization_cross_section(xl, beam_energy, 'sw')
        eff = estimate_detector_efficiency(xl, detector['t_polymer'],detector['t_Au'], detector['t_Si'], detector['t_Si_2'])
        res.append(q * w * a * eff)
    return (res[1] / res[0]) * (atomic_weight[0] / atomic_weight[1])


def _plot_model(zeta_factors, line_energies, xray_lines, detector_efficiency, offset_atomic_number):
    num = len(zeta_factors)

    # Zeta-factors by energy
    plt.figure()
    plt.plot(line_energies, zeta_factors, 'ro')
    plt.plot(known_energies, zeta_factors_known, 'ys')
    plt.ylabel("Zeta-factors")
    plt.ylabel("Zeta-factors")
    plt.xlabel("Energy (keV)")
    # Detector efficiency by energy
    plt.figure()
    plt.plot(line_energies, detector_efficiency, 'g * --')
    plt.title("Detector efficiency")
    plt.xlabel("Energy (keV)")
    # Plot by element
    elements = [i[0] for i in xray_lines][:num]
    atomic_numbers = np.arange(offset_atomic_number, len(xray_lines) + offset_atomic_number - 1)[:num]
    # Atomic number help
    PLOT_SPACE = 1
    elements_plot_labels = [el for (i, el) in enumerate(elements) if not i % PLOT_SPACE]
    atomic_numbers_plot_labels = [a for (i, a) in enumerate(atomic_numbers) if not i % PLOT_SPACE]
    # Zeta-factor by atomic number/element
#     plt.figure()
#     print(atomic_numbers)
#     print(len(zeta_factors))
#     plt.plot(atomic_numbers, zeta_factors[:len(atomic_numbers)], 'ro')
#     plt.plot(known_atomic_numbers, zeta_factors_known, 'ys')
#     plt.ylabel("Zeta-factors")
#     plt.ylabel("Zeta-factors")
#     plt.xlabel("Element")
#     plt.xticks(atomic_numbers_plot_labels, elements_plot_labels)
#     plt.grid(axis='x')
#     # Detector efficiency by atomic number
#     plt.figure()
#     plt.plot(atomic_numbers, detector_efficiency, 'g * --')
#     plt.xticks(atomic_numbers_plot_labels, elements_plot_labels)
#     plt.title("Detector efficiency")
#     plt.xlabel("Element")
#     plt.grid(axis='x')


def calculate_and_plot_model_lines(lines,s, detector, known_energies, zeta_factors_known, plot=True):
    """
    Returns a list of fitted zeta-factors for
    all elements from C to U.
    Parameters
    ----------
    s: float
    scaling factor for cross-section
    detector: dictionary
    Detector configuration: thickness of windows/layers and collection
    angle.
    known_energies: list of floats
    Energies in (keV) of the X-ray lines used for fitting
    zeta_factors_known: list of float
    The zeta-factors of the X-ray lines used for fitting.
    plot: bool
    If True, plots the model
    """

    xray_lines_K_alpha = [['C', 'C_Ka'], ['N', 'N_Ka'], ['O', 'O_Ka'], ['F',
                                                                        'F_Ka'], ['Ne', 'Ne_Ka'], ['Na', 'Na_Ka'],
                          ['Mg', 'Mg_Ka'], ['Al', 'Al_Ka'], ['Si',
                                                             'Si_Ka'], ['P', 'P_Ka'], ['S', 'S_Ka'], ['Cl', 'Cl_Ka'],
                          ['Ar', 'Ar_Ka'], ['K',
                                            'K_Ka'], ['Ca', 'Ca_Ka'], ['Sc', 'Sc_Ka'], ['Ti', 'Ti_Ka'], ['V', 'V_Ka'],
                          ['Cr',
                           'Cr_Ka'], ['Mn', 'Mn_Ka'], ['Fe', 'Fe_Ka'], ['Co', 'Co_Ka'], ['Ni', 'Ni_Ka'], ['Cu',
                                                                                                          'Cu_Ka'],
                          ['Zn', 'Zn_Ka'], ['Ga', 'Ga_Ka'], ['Ge', 'Ge_Ka'], ['As', 'As_Ka'], ['Se',
                                                                                               'Se_Ka'],
                          ['Br', 'Br_Ka'], ['Kr', 'Kr_Ka'], ['Rb', 'Rb_Ka'], ['Sr', 'Sr_Ka'], ['Y',
                                                                                               'Y_Ka'], ['Zr', 'Zr_Ka'],
                          ['Nb', 'Nb_Ka'], ['Mo', 'Mo_Ka'], ['Tc', 'Tc_Ka'], ['Ru',
                                                                              'Ru_Ka'], ['Rh', 'Rh_Ka'],
                          ['Pd', 'Pd_Ka'], ['Ag', 'Ag_Ka'], ['Cd', 'Cd_Ka'], ['In',
                                                                              'In_Ka'], ['Sn', 'Sn_Ka'],
                          ['Sb', 'Sb_Ka'], ['Te', 'Te_Ka'], ['I', 'I_Ka'], ['Xe',
                                                                            'Xe_Ka'], ['Cs', 'Cs_Ka'], ['Ba', 'Ba_Ka'],
                          ['La', 'La_Ka'], ['Ce', 'Ce_Ka'], ['Pr',
                                                             'Pr_Ka'], ['Nd', 'Nd_Ka'], ['Pm', 'Pm_Ka'],
                          ['Sm', 'Sm_Ka'], ['Eu', 'Eu_Ka'], ['Gd',
                                                             'Gd_Ka'], ['Tb', 'Tb_Ka'], ['Dy', 'Dy_Ka'],
                          ['Ho', 'Ho_Ka'], ['Er', 'Er_Ka'], ['Tm',
                                                             'Tm_Ka'], ['Yb', 'Yb_Ka'], ['Lu', 'Lu_Ka'],
                          ['Hf', 'Hf_Ka'], ['Ta', 'Ta_Ka'], ['W',
                                                             'W_Ka'], ['Re', 'Re_Ka'], ['Os', 'Os_Ka'], ['Ir', 'Ir_Ka'],
                          ['Pt', 'Pt_Ka'], ['Au',
                                            'Au_Ka'], ['Hg', 'Hg_Ka'], ['Tl', 'Tl_Ka'], ['Pb', 'Pb_Ka'],
                          ['Bi', 'Bi_Ka'], ['Po',
                                            'Po_Ka'], ['At', 'At_Ka'], ['Rn', 'Rn_Ka'], ['Fr', 'Fr_Ka'],
                          ['Ra', 'Ra_Ka'], ['Ac',
                                            'Ac_Ka'], ['Th', 'Th_Ka'], ['Pa', 'Pa_Ka'], ['U', 'U_Ka']]
    
    xray_lines_L_alpha = [['Ca', 'Ca_La'], ['Sc', 'Sc_La'], ['Ti', 'Ti_La'], ['V', 'V_La'],
                      ['Cr',
                       'Cr_La'], ['Mn', 'Mn_La'], ['Fe', 'Fe_La'], ['Co', 'Co_La'], ['Ni', 'Ni_La'], ['Cu',
                                                                                                      'Cu_La'],
                      ['Zn', 'Zn_La'], ['Ga', 'Ga_La'], ['Ge', 'Ge_La'], ['As', 'As_La'], ['Se',
                                                                                           'Se_La'],
                      ['Br', 'Br_La'], ['Kr', 'Kr_La'], ['Rb', 'Rb_La'], ['Sr', 'Sr_La'], ['Y',
                                                                                           'Y_La'], ['Zr', 'Zr_La'],
                      ['Nb', 'Nb_La'], ['Mo', 'Mo_La'], ['Tc', 'Tc_La'], ['Ru',
                                                                          'Ru_La'], ['Rh', 'Rh_La'],
                      ['Pd', 'Pd_La'], ['Ag', 'Ag_La'], ['Cd', 'Cd_La'], ['In',
                                                                          'In_La'], ['Sn', 'Sn_La'],
                      ['Sb', 'Sb_La'], ['Te', 'Te_La'], ['I', 'I_La'], ['Xe',
                                                                        'Xe_La'], ['Cs', 'Cs_La'], ['Ba', 'Ba_La'],
                      ['La', 'La_La'], ['Ce', 'Ce_La'], ['Pr',
                                                         'Pr_La'], ['Nd', 'Nd_La'], ['Pm', 'Pm_La'],
                      ['Sm', 'Sm_La'], ['Eu', 'Eu_La'], ['Gd',
                                                         'Gd_La'], ['Tb', 'Tb_La'], ['Dy', 'Dy_La'],
                      ['Ho', 'Ho_La'], ['Er', 'Er_La'], ['Tm',
                                                         'Tm_La'], ['Yb', 'Yb_La'], ['Lu', 'Lu_La'],
                      ['Hf', 'Hf_La'], ['Ta', 'Ta_La'], ['W',
                                                         'W_La'], ['Re', 'Re_La'], ['Os', 'Os_La'], ['Ir', 'Ir_La'],
                      ['Pt', 'Pt_La'], ['Au',
                                        'Au_La'], ['Hg', 'Hg_La'], ['Tl', 'Tl_La'], ['Pb', 'Pb_La'],
                      ['Bi', 'Bi_La'], ['Po',
                                        'Po_La'], ['At', 'At_La'], ['Rn', 'Rn_La'], ['Fr', 'Fr_La'],
                      ['Ra', 'Ra_La'], ['Ac',
                                        'Ac_La'], ['Th', 'Th_La'], ['Pa', 'Pa_La'], ['U', 'U_La']]

    zeta_factors = []
    line_energies = []
    detector_efficiency = []
    if lines=='K':
        xray_lines=xray_lines_K_alpha
        line='Ka'
        offset=6
    elif lines == 'L':
        xray_lines=xray_lines_L_alpha
        line='La'
        offset=20
    for element_line in xray_lines:
        energy = elements_db[element_line[0]]['Atomic_properties']['Xray_lines'][line]['energy (keV)']
        if energy > 20:
            continue
#             break
        line_energies.append(energy)
        zeta_factors.append(calculate_theoretical_zeta_factor(element_line[1], detector, 200) / s)
        detector_efficiency.append(
            estimate_detector_efficiency(element_line[1], detector['t_polymer'], detector['t_Au'], detector['t_Si'],
                                         detector['t_Si_2']))
    if plot:
        _plot_model(zeta_factors, line_energies, xray_lines,
                    detector_efficiency, offset)
    return zeta_factors, line_energies


def zeta_factor_model(x, s, s_Si, s_Au):
    det = detector_data.copy()

    det['t_Si'] *= s_Si
    det['t_Au'] *= s_Au
    return [calculate_theoretical_zeta_factor(z, det, 200) / s for z in x]


def cal_chi_sq(xray_lines, zexp, zerr, det, s):
    chi_sq = 0

    for (xl, z, err) in zip(xray_lines, zexp, zerr):
        zmodel = calculate_theoretical_zeta_factor(xl, det, 200) / s
        temp = (z - zmodel) / err
        chi_sq += temp ** 2
    return chi_sq


def fit_model_zeta_factor(xray_lines, zexp, zerr, p0=None):
    """
    Returns:
    s: The scaling factor for the ionization cross section.
    det_adj: The adjusted detector configuration
    chi_sq: The chi-squared value after the fitting
    cov: The covariance matrix after the fitting.
    """

    import scipy.optimize as optim
    res, cov = optim.curve_fit(zeta_factor_model, xdata=xray_lines, ydata=zexp, p0=None, sigma=zerr,
                               bounds=([0, 0, 0], np.inf), method='trf')
    s = res[0]  # scaling factor for ionization cross-section
    s_Si = res[1]
    s_Au = res[2]
    det_adj = detector_data.copy()
    det_adj['t_Si'] *= s_Si
    det_adj['t_Au'] *= s_Au
    chi_sq = cal_chi_sq(xray_lines, zexp, zerr, det_adj, s)
    return s, det_adj, chi_sq, cov, res

# Experimental values for plot
zeta_AsK1=706
zeta_GaK1=608
zeta_AsK2=576
zeta_GaK2=486
# zeta_AsK2=
zeta_C = 885
zeta_GeK=np.mean([729,740,745])
known_energies_k = [0.2774,9.886,10.544,9.252]
known_atomic_numbers_k = [32,33,31]#,46,79]
zeta_AsL=955
zeta_GaL=826
zeta_GeL=900
zeta_AuL=3424
zeta_PdL=1300
known_energies_l=[1.188,1.282,1.098,2.839, 9.713]

llines=['Ge_La', 'As_La', 'Ga_La','Pd_La','Au_La']
klines=['C_Ka','Ge_Ka', 'As_Ka', 'Ga_Ka']
zeta_factors_known_k = [zeta_C,zeta_GeK, zeta_AsK1, zeta_GaK1]#,zeta_Pd,zeta_Au]
# zeta_factors_known_l = [zeta_GeL, zeta_AsL, zeta_GaL,zeta_Pd,zeta_Au]

zeta_factors_known=zeta_factors_known_k
known_energies=known_energies_k
zerr = [z * 0.10 for z in zeta_factors_known]  # assume 10 % error
s, det_adj, chi_sq, cov, res = fit_model_zeta_factor(klines, zeta_factors_known, zerr)

# print(known_energies_k,zeta_factors_known)
model_zeta_factors, line_energies = calculate_and_plot_model_lines('K',s, det_adj,known_energies, zeta_factors_known)


def test_k_zeta_same_val(detector_data=detector_data, beam_energy=200):
    """ Running this will give an error if there is any inconsistency
    between the theoretical calculated zeta-factor and k-factor """
    k_AlSi = calculate_theoretical_k_factor(['Al_Ka', 'Si_Ka'], detector_data,
                                            beam_energy)
    zeta_Al = calculate_theoretical_zeta_factor('Al_Ka', detector_data,
                                                beam_energy)
    zeta_Si = calculate_theoretical_zeta_factor('Si_Ka', detector_data,
                                                beam_energy)
    k_AlSi_from_zeta = zeta_Al / zeta_Si

    np.testing.assert_allclose(k_AlSi, k_AlSi_from_zeta)

    test_k_zeta_same_val()  # Should not do anything, unless error


In [72]:
plt.close('all')